In [50]:
#Import dependencies
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from datetime import datetime
import time

In [51]:
#Set path of input file
file_path = "Resources/US_Salary_Cleaned.csv"
#Read input file into a DataFrame
salaries_df = pd.read_csv(file_path)
salaries_df

,timestamp,company,title,totalyearlycompensation,yearsofexperience,yearsatcompany,basesalary,stockgrantvalue,bonus,gender,...,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,city,state,country
0,2017-06-07 11:33:00,Oracle,Product Manager,127000,1.5,1.5,107000,20000.0,10000.0,none,...,0,0,0,0,0,0,0,Redwood City,CA,USA
1,2017-06-10 17:11:00,eBay,Software Engineer,100000,5.0,3.0,100000,0.0,0.0,none,...,0,0,0,0,0,0,0,San Francisco,CA,USA
2,2017-06-11 14:53:00,Amazon,Product Manager,310000,8.0,0.0,155000,0.0,0.0,none,...,0,0,0,0,0,0,0,Seattle,WA,USA
3,2017-06-20 10:58:00,Microsoft,Software Engineer,157000,5.0,3.0,157000,0.0,0.0,none,...,0,0,0,0,0,0,0,Mountain View,CA,USA
4,2017-06-21 17:27:00,Microsoft,Software Engineer,208000,8.5,8.5,208000,0.0,0.0,none,...,0,0,0,0,0,0,0,Seattle,WA,USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44718,2018-09-09 11:52:00,Google,Software Engineer,327000,10.0,1.0,155000,150000.0,22000.0,none,...,0,0,0,0,0,0,0,Seattle,WA,USA
44719,2018-09-13 08:23:00,Microsoft,Software Engineer,237000,2.0,2.0,146900,73200.0,16000.0,none,...,0,0,0,0,0,0,0,Redmond,WA,USA
44720,2018-09-13 14:35:00,MSFT,Software Engineer,220000,14.0,12.0,157000,25000.0,20000.0,none,...,0,0,0,0,0,0,0,Seattle,WA,USA
44721,2018-09-16 16:10:00,Salesforce,Software Engineer,280000,8.0,4.0,194688,57000.0,29000.0,none,...,0,0,0,0,0,0,0,San Francisco,CA,USA


In [52]:
#Set path of input file
file_path = "Resources/Levels_Fyi_Salary_Data.csv"
#Read input file into a DataFrame
original_df = pd.read_csv(file_path)
original_df.head()

,timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,...,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education
0,6/7/2017 11:33,Oracle,L3,Product Manager,127000,"Redwood City, CA",1.5,1.5,NaN,107000,...,0,0,0,0,0,0,0,0,NaN,NaN
1,6/10/2017 17:11,eBay,SE 2,Software Engineer,100000,"San Francisco, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN
2,6/11/2017 14:53,Amazon,L7,Product Manager,310000,"Seattle, WA",8.0,0.0,NaN,155000,...,0,0,0,0,0,0,0,0,NaN,NaN
3,6/17/2017 0:23,Apple,M1,Software Engineering Manager,372000,"Sunnyvale, CA",7.0,5.0,NaN,157000,...,0,0,0,0,0,0,0,0,NaN,NaN
4,6/20/2017 10:58,Microsoft,60,Software Engineer,157000,"Mountain View, CA",5.0,3.0,NaN,0,...,0,0,0,0,0,0,0,0,NaN,NaN


In [53]:
# Print out the city value counts
city_counts = salaries_df.city.value_counts()
print(city_counts)
#So there are 642 city/state pairs to retrieve from the API

Seattle            6945
San Francisco      5146
New York           3798
Redmond            2455
Sunnyvale          1823
                   ... 
West Palm Beach       1
Kalamazoo             1
Inwood                1
Butte                 1
Hilbert               1
Name: city, Length: 642, dtype: int64


In [54]:
print(original_df['location'].unique())

['Redwood City, CA' 'San Francisco, CA' 'Seattle, WA' ...
 'Pasir Ris, SG, Singapore' 'Richmond, VA, United States' 'Hilbert, WI']


In [55]:
original_us_df = original_df.loc[original_df['dmaid']>0]
original_us_df
locations = original_us_df['location'].unique()
locations[0]

'Redwood City, CA'

In [56]:
from config import radar_api_key
import requests
base_url = "https://api.radar.io/v1/search/autocomplete?query=" 
base_url
#radar_api_key
#"https://api.radar.io/v1/search/autocomplete?query=brooklyn+roasting&near=40.70390,-73.98670" \
  #-H "Authorization: prj_live_pk_..."

'https://api.radar.io/v1/search/autocomplete?query='

In [57]:
#test one row for api call to see if code works
url = f"{base_url}{locations[0]}"
city_url = url 
city_url
city_latlong = requests.get(city_url,headers = {"Authorization":radar_api_key}).json()
city_latlong

{'meta': {'code': 200},
 'addresses': [{'latitude': 37.52658,
   'longitude': -122.206273,
   'geometry': {'type': 'Point', 'coordinates': [-122.206273, 37.52658]},
   'country': 'United States',
   'countryCode': 'US',
   'countryFlag': '🇺🇸',
   'distance': 3907642,
   'city': 'Redwood City',
   'stateCode': 'CA',
   'state': 'California',
   'layer': 'locality',
   'formattedAddress': 'Redwood City, CA USA',
   'addressLabel': 'Redwood City'},
  {'latitude': 37.485813,
   'longitude': -122.231446,
   'geometry': {'type': 'Point', 'coordinates': [-122.231446, 37.485813]},
   'country': 'United States',
   'countryCode': 'US',
   'countryFlag': '🇺🇸',
   'distance': 3911063,
   'city': 'Redwood City',
   'number': '1',
   'postalCode': '94063',
   'stateCode': 'CA',
   'state': 'California',
   'street': 'James Ave Sequoia Station',
   'layer': 'place',
   'formattedAddress': '1 James Ave Sequoia Station, Redwood City, CA 94063 USA',
   'placeLabel': 'Redwood City Caltrain Station'},
  

In [58]:
locations

array(['Redwood City, CA', 'San Francisco, CA', 'Seattle, WA',
       'Sunnyvale, CA', 'Mountain View, CA', 'Redmond, WA',
       'Bellevue, WA', 'Menlo Park, CA', 'Cupertino, CA', 'New York, NY',
       'Los Gatos, CA', 'Baltimore, MD', 'Sacramento, CA',
       'Hillsboro, OR', 'Aliso Viejo, CA', 'San Jose, CA',
       'Palo Alto, CA', 'Kirkland, WA', 'Washington, DC', 'Austin, TX',
       'Boston, MA', 'San Diego, CA', 'New Orleans, LA',
       'Santa Clara, CA', 'Irvine, CA', 'Pittsburgh, PA', 'San Ramon, CA',
       'Los Angeles, CA', 'Folsom, CA', 'Charleston, SC',
       'Culver City, CA', 'Mc Lean, VA', 'Portland, OR', 'Boulder, CO',
       'Pleasanton, CA', 'Fremont, CA', 'Beaverton, OR', 'Omaha, NE',
       'Waltham, MA', 'Philadelphia, PA', 'Indianapolis, IN',
       'Milwaukee, WI', 'Minneapolis, MN', 'Chicago, IL', 'Herndon, VA',
       'Denver, CO', 'Albany, NY', 'Hoboken, NJ', 'Newark, NJ',
       'Chandler, AZ', 'Colorado Springs, CO', 'Costa Mesa, CA',
       'San Bruno

In [59]:
salaries_df['city, state'] = salaries_df['city'] + ", " + salaries_df['state']
salaries_df['city, state']

0         Redwood City, CA
1        San Francisco, CA
2              Seattle, WA
3        Mountain View, CA
4              Seattle, WA
               ...        
44718          Seattle, WA
44719          Redmond, WA
44720          Seattle, WA
44721    San Francisco, CA
44722        Sunnyvale, CA
Name: city, state, Length: 44723, dtype: object

In [60]:
cities = salaries_df[['city','state','city, state']]
cities

,city,state,"city, state"
0,Redwood City,CA,"Redwood City, CA"
1,San Francisco,CA,"San Francisco, CA"
2,Seattle,WA,"Seattle, WA"
3,Mountain View,CA,"Mountain View, CA"
4,Seattle,WA,"Seattle, WA"
...,...,...,...
44718,Seattle,WA,"Seattle, WA"
44719,Redmond,WA,"Redmond, WA"
44720,Seattle,WA,"Seattle, WA"
44721,San Francisco,CA,"San Francisco, CA"


In [61]:
#s_array = df[["A", "B", "C"]].to_numpy()
cities_array = cities[['city','state','city, state']].to_numpy()
cities_array

array([['Redwood City', 'CA', 'Redwood City, CA'],
       ['San Francisco', 'CA', 'San Francisco, CA'],
       ['Seattle', 'WA', 'Seattle, WA'],
       ...,
       ['Seattle', 'WA', 'Seattle, WA'],
       ['San Francisco', 'CA', 'San Francisco, CA'],
       ['Sunnyvale', 'CA', 'Sunnyvale, CA']], dtype=object)

In [62]:
from config import weather_api_key
import requests
base_url = "http://api.openweathermap.org/geo/1.0/direct?q=" 
base_url

#api form for openweather
#http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}

'http://api.openweathermap.org/geo/1.0/direct?q='

In [63]:
#test one row for api call to see if code works
url = f"{base_url}{cities_array[0][0]},{cities_array[0][1]},US&appid={weather_api_key}"
city_url = url 
city_url
city_latlong = requests.get(city_url).json()
print(city_latlong)
print(city_url)

[{'name': 'Redwood City', 'local_names': {'zh': '红木城', 'en': 'Redwood City'}, 'lat': 37.4863239, 'lon': -122.232523, 'country': 'US', 'state': 'California'}]
http://api.openweathermap.org/geo/1.0/direct?q=Redwood City,CA,US&appid=7cd5349a8a81351c676e6d427aecd0bb


In [64]:
cities_short = cities_array[0:10]

In [ ]:
#Starting URL for Weather Map API call

base_url = "http://api.openweathermap.org/geo/1.0/direct?q=" 
#api form for openweather
#http://api.openweathermap.org/geo/1.0/direct?q={city name},{state code},{country code}&limit={limit}&appid={API key}

#Create an empty list to hold the lat/long data
lat_lng = []

#Print the beginning of the logging
print("Beginning Data Retrieval   ")
print("-------------------------")

#Create counters
record_count = 1
set_count = 1


#Loop through the cities and build the city url
for city in cities_array:
    
    #Group cities in sets of 600 for logging purposes (and for not exceeding api limits)
    #if i has no remainder when divided by 600, which means we have completed a set of 600, (% is the operator for modular arithmetic)
    #then increment main counter and increment set counter
    #so this if is executed after the 600th, 1200th, etc pass, this is when we sleep for a minute
    if (record_count % 600 == 0 and record_count >= 600): 
        set_count += 1
        record_count = 1
        time.sleep(60)

    #Create an endpoint url with each city
    city_url = f"{base_url}{city[0]},{city[1]},US&appid={weather_api_key}"

    #Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    #Add one to the record count
    record_count += 1
    
    try:
        #Parse the JSON and retrieve data
        city_info = requests.get(city_url).json()
        print(city_info)
        #Parse out the needed data
        city_lat = city_info[0]["lat"]
        city_lng = city_info[0]["lon"]
        city_name = city_info[0]["name"]
        city_country = city_info[0]["country"]
        city_state = city_info[0]["state"]
        
        #append the data into the city_data list
        lat_lng.append({  "Lat": city_lat,
                          "Lng": city_lng,
                          "Name": city_name,
                          "State": city_state,
                          "Country": city_country})
        
    #if an error is experienced, skip the city    
    except:
        print("City not found. Skipping...")
        pass
    
#Indicate that the Data Loading is complete
print("-------------------------")
print("Data Retrieval Complete")
print("-------------------------")

In [ ]:
len(lat_lng)

In [42]:
len(lat_lng)

694

In [43]:
lat_lng_df = pd.DataFrame(lat_lng)
lat_lng_df.head()

,Lat,Lng,Name,State,Country
0,37.486324,-122.232523,Redwood City,California,US
1,37.779026,-122.419906,San Francisco,California,US
2,47.603832,-122.330062,Seattle,Washington,US
3,37.389389,-122.083210,Mountain View,California,US
4,47.669414,-122.123877,Redmond,Washington,US


In [44]:
#write lat_lng to a csv file
lat_lng_df.to_csv("Resources/Lat_Lng.csv",index=False)